Useful links:
 - https://stackoverflow.com/questions/19846332/python-threading-inside-a-class/19846691
 

In [1]:
# Research while working on 1020 project
import socket
import threading
import time
import pickle

In [4]:
# Constants
HEADERSIZE = 10

In [30]:
class server():
    def __init__(self, ip, port):
        self.ip = ip
        self.port = port
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.socket.bind((self.ip, self.port))
        self.socket.listen(5)
        
        self.clients = []
    
    def broadcast(self, message):
        pass
    
    def _accept_clients(self):
        while True:
            sockfd, addr = self.socket.accept()
            self.clients.append(sockfd)
            print(self.clients)
    def accept_clients(self):
        threading.Thread(target=self._accept_clients).start()

class client():
    def __init__(self, username, server_address):
        self.username = username
        self.server_address = server_address
        self.socket = socket.socket()

In [17]:
def create_message(obj):
    compressed_text = pickle.dumps(obj)
    #compressed_text = bytes(text, 'utf-8')
    length = len(compressed_text)
    return bytes(f"{length:<{HEADERSIZE}}", 'utf-8') + compressed_text

In [28]:
# Testing
test_server = server('127.0.0.1', 1234)
test_server._accept_clients()
#print(create_message([1,2,3]))


[<socket.socket fd=47, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234), raddr=('127.0.0.1', 47552)>]


KeyboardInterrupt: 